# FreelyMovingEphys modular DeepLabCut data intake
## modular_dlc_intake.ipynb

DeepLabCut points, associated timestamps, and videos are read in for any or no inputs of the possible camera types: world, eye, and topdown. Should use the 'fmephys' environment.

This script, modular_dlc_intake.ipynb, is an alternative means of accessing the underlying functions to the more high-throughput terminal interface in dlc_intake.py.

Last modified August 04, 2020

## global user inputs

### file paths

In [ ]:
# source for .avi videos, DLC .h5 files, and .csv timestamp files
global_data_path = '/Users/dylanmartins/Dropbox/sci_niell_lab/data/PreyCapture/WorldCamCohortruncsvDLCselect/'
# path into which outputs will be saved (directory will be created if it does not already exist)
global_save_path = '/Users/dylanmartins/local_workspace/analysis_outputs/PreyCapture/WorldCamOutputs/jupyterworld_test_06/'


### trial info

In [ ]:
# number of labeled topdown DeepLabCut points (including points on cricket, if there is one)
topdown_pt_num = 10
# if there is a cricket in the tank as the last two DeepLabCut points (True/False)
cricket = True
# number of labeled eye camera points (including tear duct and back of eye, if those exist)
eye_pt_num = 8
# if there are eye points labeled for the tear duct and back of the eye (True/False)
tear = False
# use BonsaiTS timestamps (True) or FlirTS timestamps (False)
bonsaitime = True
# name of trial to run on e.g. '070920_mouse01_test01' -- do not include the file extension in the name string, the extensions '.avi', '_BonsaiTS.csv', and '.h5' will be added as needed
trial_name = '112619_J463b_cricket1clip2'
# number of topdown cameras to read in; currently handles up to 3
toptrial_count = 1
# some parameters to find the pupil rotation
ranger = 10
world_interp_method = 'linear'


### thresholding and saftey

In [ ]:
# DLC likelihood threshold (default=0.99)
lik_thresh = 0.99
# max number of pixels for radius of pupil (default=50)
pxl_thresh = 50
# maximum ratio of ellipse shortaxis to longaxis (default=0.90)
ell_thresh = 0.90
# value with which to correct y-coordinates in topdown view (default=0)
coord_cor = 0


### module imports

In [ ]:
from nb_wrapper import topdown_intake, eye_intake
from util.save_data import savetrial
from util.track_world import adjust_world, find_pupil_rotation


## topdown tracking
The topdown_intake function will read in a single topdown camera view's DLC data, timestamps, and videos, threshold the values based on likelihood in an xarray format, and save a video of the eye with the ellipse plotted on it.
The function should be run on each camera seperately, and each output of topdown_intake should be saved with savetrial individually so that there is one .nc file for every topdown view (i.e. TOP1, TOP2, and TOP3).

In [ ]:
# run topdown_intake
for num in range(0,toptrial_count):
    trialext = 'TOP' + str(num)
    xtop = topdown_intake(global_data_path, top1, top_name, global_save_path, lik_thresh, coord_cor, topdown_pt_num, cricket, bonsaitime)
    savetrial(xtop, global_save_path, trial_name, trialext)


## eye tracking and worldcam adjustment
The eye_intake function will read in a single eye's DLC data, timestamps, and videos, threshold by likeihood, get out ellipse paramaters including phi and theta, and save a video of the eye with the ellipse plotted on it.

NOTE: corresponding eye view must be analyzed before running worldcam views.

### left eye

In [ ]:
LEye = trial_name + '_LEye'

# find ellipse parameters
xLEye = eye_intake(global_data_path, LEye, 'LEye', global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)
savetrial(xLEye, global_save_path, trial_name, 'LEye')


### right eye

In [ ]:
REye = trial_name + '_REye'

# find ellipse parameters
xREye = eye_intake(global_data_path, REye, 'Reye', global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)
savetrial(xREye, global_save_path, trial_name, 'REye')


### left world
Has 'left eye' block been run already?

In [ ]:
LWorld = trial_name + '_LWorld'

# find pupil rotation
find_pupil_rotation(global_data_path, file_name, 'LEye', 'TOP1', 'LWorld', xLEye, global_save_path)


### right world
Has 'right eye' block been run already?

In [ ]:
# right world
RWorld = trial_name + '_RWorld'

find_pupil_rotation(global_data_path, trial_name, 'REye', 'TOP1', 'RWorld', xREye, global_save_path, world_interp_method, ranger)


In [ ]:
# to test the new function, adjust world...
adjust_world(global_data_path, file_name, eyeext, topext, worldext, xREye, global_save_path)